# dev nearest-neighbor tracking with periodic boundary conditions
Tim Tyree<br>
6.29.2022

In [1]:
use_clear_output=True
if use_clear_output:
    from IPython.display import clear_output
    clear_output(wait=True)

import sys,os
og_stdout=sys.stdout
from lib.my_initialization import *

os.chdir(nb_dir)
from lib import *

# import cupy, cudf
# from lib.rapids_func import *

import seaborn as sns
import matplotlib.ticker as mtick

#reset matplotlib settings
import matplotlib as mpl
sns.reset_orig()
mpl.rc_file_defaults()

import warnings
warnings.simplefilter("ignore", UserWarning)
%load_ext autoreload
%autoreload 2

# define module

# dev

In [2]:
#TODO: load the data from bgmc that has pid_explicit
#HINT: its on ub in bgmc/python/data/local_results


In [3]:
#TODO: implement/dev draft of simple tracker like WJ's in physical notebook
def comp_particle_trajectories_pbc_nearest_neighbors_simple(df):
    pass
#input: df  # from df_log
#output: df_traj

In [4]:
#TODO: test simple tracker on the data by comparing 'particle' and 'pid_explicit'